# Зависимости

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

from mycsv import split_file
from dataset import load_dataset
from transform import Converter

# Загрузка модели

In [ ]:
MODEL_PATH = 'fnp_lstm_maxlen200_e25_p701.h5'
model = load_model(MODEL_PATH)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           316000    
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          82432     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 126)               16254     
Total params: 546,270
Trainable params: 546,270
Non-trainable params: 0
_________________________________________________________________


# Загрузка данных для исследования

In [ ]:
split_file('dataset.csv', split_ratio=0.8)

После разделения датасета в дериктории появиться два файла (тренировочный сет и тестовый).

* `maxlen` задает максимальную длину последовательности узлов функции (средняя длина последовательности ~50)

* `verbose` задает многословность функций обработчиков (то столько будет выводится информации во время обратоки); доступные значения от 0 до 3

In [ ]:
train_file_name = 'train_dataset.csv'
test_file_name = 'test_dataset.csv'
maxlen = 200
verbose = 0

Загрузка конвертеров из файлов.

In [ ]:
data_converter = Converter.from_file('dictionary_9875.json')
marks_converter = Converter.from_file('dictionary_126.json')

Загрузка данных для исследования.

In [ ]:
dataset = load_dataset(train_file_name, 
                       test_file_name, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_train, y_train, x_test, y_test, data_converter, marks_converter = dataset

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Доля верных ответов на тестовых данных составляет:',round(scores[1]* 100, 4),'%')

1034/1034 [==============================] - 11s 10ms/step - loss: 1.3214 - accuracy: 0.7013
Доля верных ответов на тестовых данных составляет: 70.1255 %


**Загрузка реальных сниппетов функций**

In [ ]:
SNIPPETS_PATH = 'real_snippets.csv'
snippets = load_dataset(SNIPPETS_PATH, 
                       SNIPPETS_PATH, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_snippets, y_snippets, x_test, y_test, data_converter, marks_converter = snippets

# Исследование работы модели на исходном датасете

На данном этапе следует проанализировать работу модели на примерах тестовой выборки, с целью исключения функций, которые приводят к коллизиям.

Для этого можно проанализировать следующие параметры:

1. Вероятность правильного определения функции для каждого имени.

2. Самая частотная ошибка при неправильном предсказании.

### Вероятность правильного определения функции для каждого имени.

In [ ]:
def make_prediction(model, x):
  a = np.array(x)
  a = (np.expand_dims (a, 0))
  return model.predict(a)[0]

Общее количсество функций для каждого имени.

In [ ]:
counter_names = {}
for y in y_test:
  index = np.argmax(y)
  label = marks_converter.index_to_item(index)
  counter_names[label] = counter_names.get(label, 0) + 1

Сумма вероятностей для каждого имени функции (оптимизировать).

In [ ]:
counter_prob = {}
for i, x in enumerate(x_test):
  predict = make_prediction(model, x)
  index = np.argmax(y_test[i])
  label = marks_converter.index_to_item(index)
  counter_prob[label] = counter_prob.get(label, 0) + predict[index]

Сумма всех вероятностей для одного имени деленное на количество функций в выборке дает вероятность правильного определения функции.
$$
counter\_prob_i / counter\_names_i = result_i
$$

In [ ]:
result = {}
for key, score in counter_prob.items():
  result[key] = score / counter_names[key]

In [ ]:
result_dict = [(key, value) for key, value in result_dict.items()]
result_dict.sort(key=lambda x: x[1])

In [ ]:
result_dict

[('cleanup', 0.1345981195973384),
 ('find', 0.22055030394784922),
 ('initialize', 0.24273691571900555),
 ('apply', 0.24458070610991803),
 ('append', 0.26045350122552297),
 ('dump', 0.26270877048644997),
 ('test_basic', 0.2842579569840619),
 ('info', 0.30137179788026774),
 ('match', 0.30614181874170326),
 ('count', 0.3093307565560826),
 ('check', 0.3108454771842081),
 ('log', 0.3378594057571021),
 ('load', 0.34111617746395834),
 ('get_data', 0.34198933150497146),
 ('execute', 0.3468818870488327),
 ('search', 0.3470047648041498),
 ('put', 0.3491973568334392),
 ('generate', 0.35211626954198877),
 ('reset', 0.36025666535568157),
 ('insert', 0.36156096372678803),
 ('send', 0.36723636079115063),
 ('filter', 0.3694627978718701),
 ('shutdown', 0.3745300633500129),
 ('value', 0.3746867089800447),
 ('unicode', 0.376503606672424),
 ('list', 0.3792045554949073),
 ('stop', 0.38063007923509956),
 ('test_simple', 0.38445028225110683),
 ('show', 0.3998906899524719),
 ('remove', 0.4003405003498251),
 (

# Исследование работы модели на реальных примерах

In [ ]:
#from IPython.display import Image, display
#display(Image(''))

In [ ]:
predict = make_prediction(model, x_snippets[0])

In [ ]:
index = np.argmax(predict)
label = marks_converter.index_to_item(index)
label

'post'